In [1]:
import numpy as np
from omegaconf import OmegaConf

import sys
sys.path.insert(0, '../../AgentTorch/')
from AgentTorch import Configurator

import types

In [2]:
conf = Configurator()

## Metadata

In [3]:
conf.add_metadata('num_episodes', 3)
conf.add_metadata('num_steps_per_episode', 20)
conf.add_metadata('num_substeps_per_step', 1)
conf.add_metadata('h', 72)
conf.add_metadata('w', 72)
conf.add_metadata('n_channels', 16)
conf.add_metadata('batch_size', 8)
conf.add_metadata('device', 'cpu')
conf.add_metadata('hidden_size', 128)
conf.add_metadata('fire_rate', 0.5)
conf.add_metadata('angle', 0.0)
conf.add_metadata('learning_params', {'lr': 2e-3, 'betas': [0.5, 0.5], 'lr_gamma': 0.9999, 'model_path': 'saved_model.pth'})


## Agent properties

In [4]:
w, h = conf.get('simulation_metadata.w'), conf.get('simulation_metadata.h')    
automata_number = h*w
print("Automata number: ", automata_number)

automata = conf.add_agents(key="automata", number=automata_number)

Automata number:  5184


In [5]:
## add properties to this agent
n_channels = conf.get('simulation_metadata.n_channels')
batch_size = conf.get('simulation_metadata.batch_size')
device = conf.get('simulation_metadata.device')

from models.nca.substeps.utils import nca_initialize_state

arguments_list = [conf.create_variable(key='n_channels', name="n_channels", learnable=False, shape=(1,), initialization_function=None, value=n_channels, dtype="int"),
                conf.create_variable(key='batch_size', name="batch_size", learnable=False, shape=(1,), initialization_function=None, value=batch_size, dtype="int"),
                conf.create_variable(key='device', name="device", learnable=False, shape=(1,), initialization_function=None, value=device, dtype="str")]

cell_state_initializer = conf.create_initializer(generator = nca_initialize_state, arguments=arguments_list)

In [6]:
automata_cell_state = conf.add_property(root='state.agents.automata', key='cell_state', name="cell_state", learnable=True, shape=(n_channels,), initialization_function=cell_state_initializer, dtype="float")

## Object properties

## Network

In [7]:
from AgentTorch.helpers.environment import grid_network
conf.add_network('evolution_network', grid_network, arguments={'shape': [w, h]})

## Environment

## Substep

In [8]:
from AgentTorch.models.nca.substeps.evolve_cell.transition import NCAEvolve
evolve_transition = conf.create_function(NCAEvolve, input_variables={'cell_state':'agents/automata/cell_state'}, output_variables=['cell_state'], fn_type="transition")

/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so, 6): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  Referenced from: /Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so
  Reason: image not found
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: dlopen(/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so, 6): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  

In [9]:
automata = conf.get('state.agents.automata')
conf.add_substep(name="Evolution", active_agents=["automata"], transition_fn=evolve_transition)

In [10]:
conf.render('test_config.yaml')